# BABC-implementacija

Implementacija u potpunosti prati pseudo kod iz [1]. Jedina razlika je vezana uz računanje evaluacije klasifikacijskog algoritma, tj. funkcije dobrote u pčelinjem algoritmu.

In [1]:
# Importing the libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

import random

from sklearn.metrics import balanced_accuracy_score

In [2]:
def KNN_Classifier(X_train_features, X_test_features, y_train, y_test, n):
    classifier = KNeighborsClassifier(n_neighbors = n, metric = 'minkowski', p = 2)
    classifier.fit(X_train_features, y_train)
    
    y_pred = classifier.predict(X_test_features)
    
    return y_pred

In [3]:
def RF_Classifier(X_train_features, X_test_features, y_train, y_test, n):
    classifier = RandomForestClassifier(n_estimators = n, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_features, y_train)
    
    y_pred = classifier.predict(X_test_features)
    
    return y_pred

In [4]:
def NB_Classifier(X_train_features, X_test_features, y_train, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train_features, y_train)
    
    y_pred = classifier.predict(X_test_features)
    
    return y_pred

Evaluacijska metrika je promijenjena zbog *accuracy paradox*-a objašnjenog u [2]. Koristi se *balanced_accuracy* pojašnjena u [3].

In [5]:
def calculate_accuracy(bee, X_train, X_test, y_train, y_test, algorithm, n):
    #count number of features
    k=0
    for j in range(0,len(bee)):
        if bee[j]==1:
            k+=1
    X_train_features = np.zeros((X_train.shape[0], k))
    X_test_features = np.zeros((X_test.shape[0], k))
        
    k=0
    for j in range(0,len(bee)):
        if bee[j]==1:
            X_train_features[:,k]=X_train[:,j]
            X_test_features[:,k]=X_test[:,j]
            k+=1
    
    if(algorithm == 'KNN'):
        y_pred = KNN_Classifier(X_train_features, X_test_features, y_train, y_test, n)
    elif(algorithm == 'RF'):
        y_pred = RF_Classifier(X_train_features, X_test_features, y_train, y_test, n)
    elif(algorithm == 'NB'):
        y_pred = NB_Classifier(X_train_features, X_test_features, y_train, y_test)
    else:
        raise ValueError('Key word is not recognized.')
        
    return balanced_accuracy_score(y_test, y_pred)

In [6]:
def init(number_employed_bees, columns_number, employed_matrix):
    for i in range(0,number_employed_bees):
        test=0
        while test==0:
            array = np.zeros(columns_number)
            for j in range(0,columns_number):
                rand_number = random.random()
                if rand_number <= 0.5:
                    array[j]=0
                else:
                    array[j]=1
            if not np.isin(1, array):
                continue
            for j in range(0,i):
                if np.array_equal(employed_matrix[j],array):
                    continue
            employed_matrix[i] = array
            test=1

In [7]:
def init_accuracies_and_best(number_employed_bees, accuracies, X_train, X_test, y_train, y_test, algorithm, n, best, best_bee, 
                             employed_matrix, columns_number):
    for i in range(0,number_employed_bees):
        accuracies[i]=calculate_accuracy(employed_matrix[i], X_train, X_test, y_train, y_test, algorithm, n)

    best = np.amax(accuracies)
    for i in range(number_employed_bees):
        if accuracies[i] == best:
            best_bee = [employed_matrix[i][k] for k in range(columns_number)]

In [8]:
def iterations(max_iterations_number, limit_array, limit, employed_matrix, number_employed_bees, columns_number, algorithm, n,
               accuracies, number_onlooker_bees, X_train, X_test, y_train, y_test, best, best_bee):
    for i in range(0,max_iterations_number):
        limit_array= [x+1 for x in limit_array]
        for j in range(0,number_employed_bees):
            bee = [employed_matrix[j][k] for k in range(columns_number)] 
            first = np.random.randint(0,columns_number)
            second = np.random.randint(0,columns_number) 
            while first==second or bee[first]==bee[second]:
                second = np.random.randint(0,columns_number)
            
            temp=bee[first]
            bee[first]=bee[second]
            bee[second]=temp
            bee_accuracy = calculate_accuracy(bee, X_train, X_test, y_train, y_test, algorithm, n)
            
            if bee_accuracy > accuracies[j]:
                    employed_matrix[j]=bee
                    accuracies[j]=bee_accuracy
                    limit_array[j]=0

        P = np.zeros(number_employed_bees)
        numerator = np.sum(accuracies)
        for j in range(0,number_employed_bees):
            P[j]=accuracies[j]/numerator

        onlooker_bees_indexes = np.zeros(number_onlooker_bees)
        k=0    
        for j in range(0,number_onlooker_bees):
            pick = random.random() 
            if pick < P[0]:
                onlooker_bees_indexes[k]=0
                k+=1
                continue
            for l in range(1,number_employed_bees):
                if pick < np.sum(P[0:l+1]) and pick >= np.sum(P[0:l]):
                    onlooker_bees_indexes[k]=l
                    k+=1
                    break

        for j in onlooker_bees_indexes:
            bee = [employed_matrix[int(j)][k] for k in range(columns_number)]
            first = np.random.randint(0,columns_number)
            second = np.random.randint(0,columns_number)
            while first==second or bee[first]==bee[second]:
                second = np.random.randint(0,columns_number)
            temp=bee[first]
            bee[first]=bee[second]
            bee[second]=temp
            bee_accuracy = calculate_accuracy(bee, X_train, X_test, y_train, y_test, algorithm, n)
            if bee_accuracy > accuracies[int(j)]:
                    employed_matrix[int(j)]=bee
                    accuracies[int(j)]=bee_accuracy
                    limit_array[int(j)]=0

        for j in range(0,number_employed_bees):
            if limit_array[j] >= limit:
                test=0      
                while test==0:
                    array = np.zeros(columns_number)
                    for k in range(0,columns_number):
                        rand_number = random.random()
                        if rand_number <= 0.5:
                            array[k]=0
                        else:
                            array[k]=1
                    if not np.isin(1, array):
                        continue
                    for k in range(0,number_employed_bees):
                        if k!=j and np.array_equal(employed_matrix[k],array):
                            continue
                    employed_matrix[j] = array
                    limit_array[j]=0
                    accuracies[j]=calculate_accuracy(array, X_train, X_test, y_train, y_test, algorithm, n)
                    test=1
          
        best_ = np.amax(accuracies)
        if best_ > best:
            best = best_
            for i in range(number_employed_bees):
                if accuracies[i] == best:
                    best_bee = [employed_matrix[i][k] for k in range(columns_number)] 
                    break
    
    return best, best_bee

# Literatura

[1] [B. Subanya, R. R. Rajalaxmi, A Novel Feature Selection Algorithm for Heart Disease
Classification, International Journal of Computational Intelligence and Informatics,
Vol. 4, No. 2, 2014.](https://www.periyaruniversity.ac.in/ijcii/issue/Vol4No2September2014/IJCII-4-2-144.pdf) <br>

[2] [8 Tactics to Combat Imbalanced Classes in Your Machine Learning Dataset](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)

[3] [Balanced accuracy: what and why?](http://mvpa.blogspot.com/2015/12/balanced-accuracy-what-and-why.html)